In [ ]:
#

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
os.chdir(path) #podanie ścieżki do katalogu z plikami źródłowymi

In [ ]:
are_R_1=pd.read_excel(file_R_1, sheet_name=0) #import dwóch plików przygotowanych przez biling z danego roku R i roku R-1 (format excel)
are_R=pd.read_excel(file_R, sheet_name=0)

In [ ]:
are_R_1=are_R_1.iloc[:,0:12] #wycięcie wymaganych kolumn

In [ ]:
are_R_1.shape

In [ ]:
are_R.shape

In [ ]:
#are_R.head() #podgląd

In [ ]:
merged=are_R.merge(are_R_1, how='left', on="Obiekt") #złączenie plików z roku R i R-1

In [ ]:
merged=merged.drop(merged.columns[[7,8,9,10,11,12,13]], axis=1) #usunięcie niepotrzebnych kolumn

In [ ]:
merged.head()

In [ ]:
#uproszczenie nazw kolumn
merged.columns=["Id_Odbiorcy","Nazwa", "Obiekt", "taryfa", "sektor", "kwh", "m3", "gmina", "powiat", "wojewodztwo", "kod_gus"]

In [ ]:
merged['licznik']=1 #dodanie kolumny, która przyda się przy grupowaniu do zliczenia ilości wystąpień

In [ ]:
merged.head()

In [ ]:
#rozdzial tabeli na tabele, gdzie gaz dostarczany jest do klientow i na tabele, gdzie zuzycie gazu jest na potrzeby wlasne
merged_bez_n=merged[merged['sektor']!='nie uwzględnione w raporcie']
merged_tylko_n=merged[merged['sektor']=='nie uwzględnione w raporcie']

In [ ]:
#grupowanie tabeli - gaz dostarczony do klientow
grouped_bez_n=merged_bez_n.groupby(['kod_gus', 'sektor']).sum()
grouped_bez_n=grouped_bez_n.reset_index()

In [ ]:
#grupowanie tabeli - gaz zuzyty na potrzeby wlasne
grouped_tylko_n=merged_tylko_n.groupby(['kod_gus', 'sektor']).sum()
grouped_tylko_n=grouped_tylko_n.reset_index()

# Transformacja danych - gaz do klientów

In [ ]:
grouped_bez_n['sektor'].value_counts() #sprawdzenie ilości wystąpień wierszy dla poszczegolnych sektorow

In [ ]:
grouped_bez_n.head()

In [ ]:
grouped_bez_n_02=grouped_bez_n[grouped_bez_n['sektor']=='02']
grouped_bez_n_03=grouped_bez_n[grouped_bez_n['sektor']=='03']
grouped_bez_n_04=grouped_bez_n[grouped_bez_n['sektor']=='04']

In [ ]:
#final_bez_n - tabela, w której w kolumnie "kod_gus" znajdą się wartości bez powtórzeń
final_bez_n=pd.DataFrame(columns=range(0,26))

In [ ]:
final_bez_n[0]=grouped_bez_n['kod_gus'] #przepisanie do finalnej tabeli

In [ ]:
final_bez_n[1]="A" #wypełnienie zgodnie z wytycznymi GUS

In [ ]:
final_bez_n.head()

In [ ]:
final_bez_n=final_bez_n.drop_duplicates(0) #usunięcie duplikatów

In [ ]:
#dodanie do każdego kod_gus danych z sektorów, które występują pod danym kodem
final_bez_n=final_bez_n.merge(grouped_bez_n_02, how='left', left_on=0, right_on='kod_gus')
final_bez_n=final_bez_n.merge(grouped_bez_n_03, how='left', left_on=0, right_on='kod_gus')
final_bez_n=final_bez_n.merge(grouped_bez_n_04, how='left', left_on=0, right_on='kod_gus')

In [ ]:
#sektor 04
final_bez_n[11].fillna(final_bez_n['licznik'], inplace=True)
final_bez_n[20].fillna(final_bez_n['kwh']/1000, inplace=True)

In [ ]:
#sektor 03
final_bez_n[16].fillna(final_bez_n['licznik_y'], inplace=True)
final_bez_n[23].fillna(final_bez_n['kwh_y']/1000, inplace=True)

In [ ]:
#sektor 02
final_bez_n[16].fillna(final_bez_n['licznik_x'], inplace=True)
final_bez_n[23].fillna(final_bez_n['kwh_x']/1000, inplace=True)

In [ ]:
final_bez_n.fillna(0, inplace=True) #wypełnienie pozycji NaN zerami zgodnie z wytycznymi GUS

In [ ]:
final_bez_n=final_bez_n.iloc[:,0:26]

In [ ]:
#wypełnianie poszczególnych kolumn według wytycznych GUS
final_bez_n[7]=final_bez_n[11]+final_bez_n[15]+final_bez_n[16]+final_bez_n[17]+final_bez_n[18] #sumowanie liczby odbiorców
final_bez_n[8]=final_bez_n[9]+final_bez_n[15]+final_bez_n[16]+final_bez_n[17] #sumowanie liczby odbiorców bez hurtu
final_bez_n[9]=final_bez_n[11]+final_bez_n[13]
final_bez_n[19]=final_bez_n[20]+final_bez_n[22]+final_bez_n[23]+final_bez_n[24]+final_bez_n[25]

In [ ]:
final_bez_n.iloc[:,2:19]=final_bez_n.iloc[:,2:19].astype(int)

In [ ]:
col_numbers=[19,20,23]
for c in col_numbers:
    final_bez_n.iloc[:,c]=final_bez_n.iloc[:,c].round(1).astype(str).apply(lambda x: x.replace(".",","))

In [ ]:
final_bez_n.head(100)

In [ ]:
#zapisanie danych do pliku *.csv gotowego do importu do systemu GUSu
final_bez_n.iloc[:,0:26].to_csv("Dzial1_Sprzedaz_do_klientow.csv", sep=";", index=False)

# Transformacje danych - zużycie własne

### Operacje w tej części są analogiczne do tych przeprowadzanych powyżej

In [ ]:
grouped_tylko_n['sektor'].value_counts()

In [ ]:
grouped_tylko_n.head()

In [ ]:
final_tylko_n=pd.DataFrame(columns=range(0,6))

In [ ]:
final_tylko_n

In [ ]:
final_tylko_n[0]=grouped_tylko_n['kod_gus']

In [ ]:
final_tylko_n[1]="A"

In [ ]:
final_tylko_n.head()

In [ ]:
final_tylko_n=final_tylko_n.drop_duplicates(0)

In [ ]:
final_tylko_n=final_tylko_n.merge(grouped_tylko_n, how='left', left_on=0, right_on='kod_gus')

In [ ]:
final_tylko_n[4].fillna(final_tylko_n['kwh']/1000, inplace=True)

In [ ]:
final_tylko_n[2]=final_tylko_n[3]+final_tylko_n[4]+final_tylko_n[5]

In [ ]:
final_tylko_n.fillna(0, inplace=True)

In [ ]:
final_tylko_n=final_tylko_n.iloc[:,0:6]

In [ ]:
col_numbers=[2,4]
for c in col_numbers:
    final_tylko_n.iloc[:,c]=final_tylko_n.iloc[:,c].round(1).astype(str).apply(lambda x: x.replace(".",","))

In [ ]:
final_tylko_n.to_csv("Dzial2_Zuzycie_wlasne.csv", sep=";", index=False)
final_tylko_n